#  <center> Problem Set 5 <center>
<center> Spring 2021 <center>
<center> 3.100/3.322, 10.402/10.602, 20.301/20.401 <center>
<center> Due:10 pm ET on Thursday, May 6, 2021 <center>

# Part 2 Image Segmentation

In [28]:
import os 
from glob import glob
import PIL
from PIL import Image
from skimage import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Download and unzip data

In [3]:
!wget -O part2data.zip https://github.com/wwang2/ML4MolEng/blob/master/psets/ps5/part2_data.zip?raw=true
! rm -r segmentation_data
! mkdir segmentation_data/
! unzip part2data.zip -d segmentation_data

--2021-04-25 17:30:37--  https://github.com/wwang2/ML4MolEng/blob/master/psets/ps5/part2_data.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/wwang2/ML4MolEng/raw/master/psets/ps5/part2_data.zip [following]
--2021-04-25 17:30:37--  https://github.com/wwang2/ML4MolEng/raw/master/psets/ps5/part2_data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps5/part2_data.zip [following]
--2021-04-25 17:30:37--  https://raw.githubusercontent.com/wwang2/ML4MolEng/master/psets/ps5/part2_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connecte

  inflating: segmentation_data/part2_segdata/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe/images/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe.png  
   creating: segmentation_data/part2_segdata/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe/masks/
  inflating: segmentation_data/part2_segdata/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe/masks/a170f409bbab5056d5910ad68cf77e866efe4be64c779b85509d6559f3c81abf.png  
  inflating: segmentation_data/part2_segdata/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe/masks/741ba3ad9827ed4f7cbb25c366d483a945b07d27dbb1d4f63463ad423b33069f.png  
  inflating: segmentation_data/part2_segdata/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe/masks/dae47f3cc68ad81bebe1974a14698bc386d41c9f5e80760fd811db46a82054bf.png  
  inflating: segmentation_data/part2_segdata/0bda515e370294ed94efd36bd53782288acacb040c171df2ed97fd691fc9d8fe/masks/8213bd500669c5cdad

   creating: segmentation_data/part2_segdata/5b12df18e4ae4df5af06052584cf0e6bef58ee2a220653890636eef88a944e14/masks/
  inflating: segmentation_data/part2_segdata/5b12df18e4ae4df5af06052584cf0e6bef58ee2a220653890636eef88a944e14/masks/4cec80de9c629bf1a622ecceff48e00cef56a474d0eb3ed37b5f177307151098.png  
  inflating: segmentation_data/part2_segdata/5b12df18e4ae4df5af06052584cf0e6bef58ee2a220653890636eef88a944e14/masks/8599063d6b3d23bb7f3c890268e2d645d5aff63c78248cb76129f8398978126b.png  
  inflating: segmentation_data/part2_segdata/5b12df18e4ae4df5af06052584cf0e6bef58ee2a220653890636eef88a944e14/masks/5451d92ea6fa7c2d28dd587bc618b0d5759f2fe20a2eaacf4d6489f767f3cc42.png  
  inflating: segmentation_data/part2_segdata/5b12df18e4ae4df5af06052584cf0e6bef58ee2a220653890636eef88a944e14/masks/5f0fd7360c4560588f0c8835461c9365d8b177ba439c62f72ef2ee9de7172e80.png  
  inflating: segmentation_data/part2_segdata/5b12df18e4ae4df5af06052584cf0e6bef58ee2a220653890636eef88a944e14/masks/8eb04bf6a0fe572f4a5

  inflating: segmentation_data/part2_segdata/c75139ef0546d2240b37afb3219eb74a06b7977818697d5c3138796472483af3/masks/66bae248859fcf1dcc3a701317146d6ef9facf536112070f6f542f5da4a682d9.png  
  inflating: segmentation_data/part2_segdata/c75139ef0546d2240b37afb3219eb74a06b7977818697d5c3138796472483af3/masks/fd36ccb1ef8e5c28884a3242c67d6c58f65f84c9aabd525c652d6f655b0001df.png  
  inflating: segmentation_data/part2_segdata/c75139ef0546d2240b37afb3219eb74a06b7977818697d5c3138796472483af3/masks/45bce8330c5d0cc5f725986a3178b7ee2337ecfc739fcaf0074f9f2831f12ac4.png  
  inflating: segmentation_data/part2_segdata/c75139ef0546d2240b37afb3219eb74a06b7977818697d5c3138796472483af3/masks/d8b25f4d3ad33424c61e5dcd92c191bc4b747a99bf387753db54f19b16043c31.png  
  inflating: segmentation_data/part2_segdata/c75139ef0546d2240b37afb3219eb74a06b7977818697d5c3138796472483af3/masks/15ec6c7a327d378229bb8e685457326c9d95ceff58eb704f118a2dbaf4444216.png  
  inflating: segmentation_data/part2_segdata/c75139ef0546d2240b37

   creating: segmentation_data/part2_segdata/29dd28df98ee51b4ab1a87f5509538ecc3e4697fc57c40c6165658f61b0d8e3a/masks/
  inflating: segmentation_data/part2_segdata/29dd28df98ee51b4ab1a87f5509538ecc3e4697fc57c40c6165658f61b0d8e3a/masks/449f157762478e3a87ecd37ac58f4fe18f037e311e4345702a4e59d347b3056f.png  
  inflating: segmentation_data/part2_segdata/29dd28df98ee51b4ab1a87f5509538ecc3e4697fc57c40c6165658f61b0d8e3a/masks/910bf3958c5d4e1df74530d550a88cb6f38631efb3515155134d10b6e91fb258.png  
  inflating: segmentation_data/part2_segdata/29dd28df98ee51b4ab1a87f5509538ecc3e4697fc57c40c6165658f61b0d8e3a/masks/98f772233f7109ccda7f14ab6e8d4ae951415c87e76233833ad39d23f3614585.png  
  inflating: segmentation_data/part2_segdata/29dd28df98ee51b4ab1a87f5509538ecc3e4697fc57c40c6165658f61b0d8e3a/masks/5a186e726a00128b1dfbeacb2d737aaedc51eda1c6a6d1ac9234363c1189bed6.png  
  inflating: segmentation_data/part2_segdata/29dd28df98ee51b4ab1a87f5509538ecc3e4697fc57c40c6165658f61b0d8e3a/masks/ac8f050f439c4d3a8d9

  inflating: segmentation_data/part2_segdata/c04fa1a74a980d790ba6f3e595fd9851f14370bb71c7cbb7846c33ca9d72687f/masks/cc027ebce487edf61846a4df14b4027920a9668dde0095a2088a656e550fa841.png  
  inflating: segmentation_data/part2_segdata/c04fa1a74a980d790ba6f3e595fd9851f14370bb71c7cbb7846c33ca9d72687f/masks/1074dc1d917424b560192485ba778a6405137118e3019bce362979c4e2560ea8.png  
  inflating: segmentation_data/part2_segdata/c04fa1a74a980d790ba6f3e595fd9851f14370bb71c7cbb7846c33ca9d72687f/masks/d28261b04bd843ad0b291334f188e4f63ecab2b4f92e9ed52b58cb8f45ce869b.png  
  inflating: segmentation_data/part2_segdata/c04fa1a74a980d790ba6f3e595fd9851f14370bb71c7cbb7846c33ca9d72687f/masks/a01d452caa6012bc825f4e7a327a5395f8fbd2ca421a54d520940da39d272ad8.png  
  inflating: segmentation_data/part2_segdata/c04fa1a74a980d790ba6f3e595fd9851f14370bb71c7cbb7846c33ca9d72687f/masks/1b4a1ba1ae04161dd0972261fe211b67b005ab75b32a03f346eba82ca5bdda2f.png  
  inflating: segmentation_data/part2_segdata/c04fa1a74a980d790ba6

  inflating: segmentation_data/part2_segdata/b8fdc02d915206bb2564e1f7da962f2b9d9d491b11afa00a76622b7932366480/masks/ce9b38f2904bc8a554ce7f18cd9dd4f0743a863d80dead5261ebdeef20a09c7e.png  
  inflating: segmentation_data/part2_segdata/b8fdc02d915206bb2564e1f7da962f2b9d9d491b11afa00a76622b7932366480/masks/dfbefccf74a6779cf83dd70df6f064af80f88366d25ca8f87dd2fb27c85ddcbb.png  
  inflating: segmentation_data/part2_segdata/b8fdc02d915206bb2564e1f7da962f2b9d9d491b11afa00a76622b7932366480/masks/3485834b6a317f706a5f22b13b0bdfb3a31bd2d21082fed8c37d28d982704fcf.png  
  inflating: segmentation_data/part2_segdata/b8fdc02d915206bb2564e1f7da962f2b9d9d491b11afa00a76622b7932366480/masks/ea5611c108b89783cea9c1dbca6eaeb3aa796cfb0cdb24b3bc7097b84d0c8477.png  
  inflating: segmentation_data/part2_segdata/b8fdc02d915206bb2564e1f7da962f2b9d9d491b11afa00a76622b7932366480/masks/b6b7d6fd318eced930062eacc417fb82e4e76383e4b4557985a7d9bd2714b8f5.png  
   creating: segmentation_data/part2_segdata/7f4d7b549d0f1a110191

  inflating: segmentation_data/part2_segdata/a0de55384fada5cbc46bd7a41f6feeef93b67d088497c7316079ccec39c2a834/masks/9571e53a54f9c2d40dfcb5602fd21f7642d7cd1fae5024f6966e5ed5a922df78.png  
  inflating: segmentation_data/part2_segdata/a0de55384fada5cbc46bd7a41f6feeef93b67d088497c7316079ccec39c2a834/masks/6f728ec6dc3596b22dc1b772f506faef20e7d9791d7f68c8aed25df98a09dad7.png  
  inflating: segmentation_data/part2_segdata/a0de55384fada5cbc46bd7a41f6feeef93b67d088497c7316079ccec39c2a834/masks/272bd9104e1a69a2d2dfca491ca5ca3a015df925a6b2eece3277e979a163897e.png  
  inflating: segmentation_data/part2_segdata/a0de55384fada5cbc46bd7a41f6feeef93b67d088497c7316079ccec39c2a834/masks/a7775fbe903e5d47d1b4473e2fc6a0160ce97d24acfa950d594aa8dfb81da384.png  
  inflating: segmentation_data/part2_segdata/a0de55384fada5cbc46bd7a41f6feeef93b67d088497c7316079ccec39c2a834/masks/9fcd22b3890f78ee4f6d7dfae97a0ef517eeed6317f0969d4057034839420012.png  
  inflating: segmentation_data/part2_segdata/a0de55384fada5cbc46b

  inflating: segmentation_data/part2_segdata/b82548ab19466b461614e6055aaf49fbc24c03a2d20e65575b680c7c28268807/masks/34ca35196e4a3349e902010e9c860a634b28047302e73f6ec8242a2e126fc887.png  
  inflating: segmentation_data/part2_segdata/b82548ab19466b461614e6055aaf49fbc24c03a2d20e65575b680c7c28268807/masks/4fa35cf2dded30ea98d79d4fe99543141b3bc1416fb631c06df76cf8fb7fd561.png  
  inflating: segmentation_data/part2_segdata/b82548ab19466b461614e6055aaf49fbc24c03a2d20e65575b680c7c28268807/masks/3e62c8eac1514b261e1700796da67d5a5be17552bc087f8e9e2c651a08c0a859.png  
  inflating: segmentation_data/part2_segdata/b82548ab19466b461614e6055aaf49fbc24c03a2d20e65575b680c7c28268807/masks/be546be52258b858a325643ff846b9b437aa4f3a9fa129c0918e2806922298c5.png  
  inflating: segmentation_data/part2_segdata/b82548ab19466b461614e6055aaf49fbc24c03a2d20e65575b680c7c28268807/masks/2e09235cf35ff7cea1b937f7318e85d60267f8edc62d8689f2b012440f77e408.png  
   creating: segmentation_data/part2_segdata/fadeb0ab092833f27daa

In [8]:
# get all image file paths
paths = glob("segmentation_data/part2_segdata/*")

In [23]:

def parse_data(path):

    # define data folders
    x_path = os.path.join(path, "images/")
    y_path = os.path.join(path, "masks/")

    # get all data paths 
    x_file = glob(os.path.join(x_path, "*.png"))[0]
    y_files = glob(os.path.join(y_path, "*.png"))

    # parse in data
    x = np.array(Image.open(x_file))
    
    # collect masks
    masks = np.array([np.array(Image.open(y)) for y in y_files])
    y = np.zeros_like(masks[0])
    for y_raw in masks:
        y = np.maximum(y, y_raw)

    # normalize
    x = x / 255.0
    y = y / 255.0

    return x, y[..., None]

In [24]:
# load one image 
x, y = parse_data(paths[0])
print(x.shape, y.shape)

(256, 256, 4) (256, 256, 1)


### 2.1 Build data-sets and data-loaders

Define Image Datasets 

Is it necessary to apply random translation to your image?  Briefly justify your answer

### 2.2 Train a U-net model that performs image segmentation

In [26]:

class DownSampling(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, name=None):
        super(DownSampling, self).__init__()

        self.conv = ConvBlock(in_channels, out_channels, kernel_size)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):

        conv_out = self.conv(x)
        output = self.max_pool(conv_out)

        return output, conv_out


class UpSampling(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, name=None):
        super(UpSampling, self).__init__()

        self.conv = ConvBlock(in_channels, out_channels, kernel_size)
        self.conv_t = nn.ConvTranspose2d(out_channels, out_channels, kernel_size, \
                                         padding=1, stride=2, output_padding=1)


    def forward(self, x, skip):

        conv_out = self.conv(x)
        output = self.conv_t(conv_out)

        output += skip

        return output


class ConvBlock(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding=1, stride=1, name=None):
        super(ConvBlock, self).__init__()

        block = []
        # first conv layer
        block.append(nn.Conv2d(in_channels, out_channels, kernel_size, \
                               padding=padding, stride=stride))
        block.append(nn.ReLU())
        block.append(nn.BatchNorm2d(out_channels))

        # second conv layer
        block.append(nn.Conv2d(out_channels, out_channels, kernel_size, \
                               padding=padding, stride=stride))
        block.append(nn.ReLU())
        block.append(nn.BatchNorm2d(out_channels))

        # make sequential
        self.conv_block = nn.Sequential(*block)


    def forward(self, x):

        output = self.conv_block(x)

        return output
    
    
class UNet(nn.Module):

    def __init__(self, num_kernel=8, kernel_size=3, dim=4, target_dim=1):
        """UNet

        Inputs:
            num_kernel: int
                number of kernels to use for the first layer
            kernel_size: int
                size of the kernel for the first layer
            dims: int
                number of color channels for input images 
            target_dim: int 
                number of channels for the output mask
        """

        super(UNet, self).__init__()

        self.num_kernel = num_kernel
        self.kernel_size = kernel_size
        self.dim = dim
        self.target_dim = 1

        # encode
        self.encode_1 = DownSampling(self.dim, num_kernel, kernel_size)
        self.encode_2 = DownSampling(num_kernel, num_kernel*2, kernel_size)
        self.encode_3 = DownSampling(num_kernel*2, num_kernel*4, kernel_size)
        self.encode_4 = DownSampling(num_kernel*4, num_kernel*8, kernel_size)

        # bridge
        self.bridge = nn.Conv2d(num_kernel*8, num_kernel*16, kernel_size, padding=1, stride=1)

        # decode
        self.decode_4 = UpSampling(num_kernel*16, num_kernel*8, kernel_size)
        self.decode_3 = UpSampling(num_kernel*8, num_kernel*4, kernel_size)
        self.decode_2 = UpSampling(num_kernel*4, num_kernel*2, kernel_size)
        self.decode_1 = UpSampling(num_kernel*2, num_kernel, kernel_size)

        self.segment = nn.Conv2d(num_kernel, self.target_dim, 1, padding=0, stride=1)
        self.activate = nn.Sigmoid()


    def forward(self, x):

        x, skip_1 = self.encode_1(x)
        x, skip_2 = self.encode_2(x)
        x, skip_3 = self.encode_3(x)
        x, skip_4 = self.encode_4(x)

        x = self.bridge(x)

        x = self.decode_4(x, skip_4)
        x = self.decode_3(x, skip_3)
        x = self.decode_2(x, skip_2)
        x = self.decode_1(x, skip_1)

        x = self.segment(x)

        pred = self.activate(x)

        return pred


    def args_dict(self):
        """model arguments to be saved
        """

        model_args = {'dim': self.dim,
                      'target_dim': self.target_dim,
                      'num_kernel' : self.num_kernel,
                      'kernel_size' : self.kernel_size}

        return model_args


In [27]:
# example model usage 

model = UNet()
y = model(torch.randn(1, 4, 256, 256))

print(y.shape)

torch.Size([1, 1, 256, 256])


implement DICE loss function 

In [ ]:
def dice_loss(pred, target):                          
    ######### You implementation #########
        
        
    return 0.0

In [37]:
def plot_seg(image, pred_seg, true_seg, mask_cutoff=0.1):

    """ Visualize segmentation results
        Inputs:
            image: orginal image, shape: 256 x 256 x 4
            pred_seg: predicted mask, shape: 256 x 256 x 1 or 256 x 256 
            true_seg: true mask, shape: 256 x 256 x 1 or 256 x 256
            mask_cutoff: if the maks values is larger than mask_cutoff, the mask will appear on the image
        """
    
    fig, ax = plt.subplots(1, 3, sharex='col', sharey='row')
    fig.set_size_inches((15,15))
    
    pred_seg = pred_seg.squeeze()
    true_seg = pred_seg.squeeze()

    ax[0].set_title("original image")
    ax[1].set_title("prediction")
    ax[2].set_title("ground truth")

    pred_mask = np.zeros((256, 256))
    pred_mask[pred_seg > mask_cutoff] = 1
    y_mask = np.zeros((256, 256))
    y_mask[true_seg > mask_cutoff] = 1

    io.imshow(image, ax=ax[0])
    io.imshow(color.label2rgb(pred_mask,image,colors=[(255,0,0)],alpha=0.0025, bg_label=0, bg_color=None), ax=ax[1])
    io.imshow(color.label2rgb(y_mask,image,colors=[(255,0,0)],alpha=0.0025, bg_label=0, bg_color=None), ax=ax[2])

    plt.show()

Train your model

Show segmentation results for 3 images from test dataset